In [79]:
# BASE FILES
gdb_str = 'klamath_d_drive2_gdb'
gdb_pro = pl.loc[gdb_str,'fp_gdb']
gdb_inv = pl.loc[gdb_str,'fname_csv']
inventory_dir = pl.loc[gdb_str,'inventory_dir']
gdb_inv
df = pd.read_csv(gdb_inv, index_col = 'ITEM')


In [80]:
feats = ['siskiyou_co_parcels_clipped','parcels_master_rectified_clipped']
fp_target = df.loc[feats[1],'DATA_LOCATION_MCMILLEN_JACOBS']
fp_source = df.loc[feats[0],'DATA_LOCATION_MCMILLEN_JACOBS']
fp_target = ol_translate(fp_target, gdb_str, True)
fp_source = ol_translate(fp_source, gdb_str, True)
fp_target = ol_translate(fp_target, gdb_str, True)

there
there


'D:\\box_offline\\klamath\\GIS_Request_Tracking\\arcpro_maps\\klamath_d_drive2\\klamath_d_drive2.gdb\\parcels\\parcels_master_rectified_clipped'

In [86]:
# FIELD MAPPINGS
# From ddp_index Notebook on May 2022; Tried on Oct 2022 for this endeavor and it works.  
# But turned out merging was the incorrect approach
# Leaving here just for future fixing.  Does NOT work. 
import compare_data
importlib.reload(sys.modules['compare_data'])
fp_source = 'parcels_master_rectified_clipped'
fp_target = 'siskiyou_co_parcels_clipped'
csv = r'C:\Users\uhlmann\Box\GIS\Project_Based\Klamath_River_Renewal_MJA\GIS_Data\data_inventory_and_tracking\working_datasets\field_mapping_tables\parcels_master_rectified_fld_map.csv'
fm = compare_data.field_mapping_retain_src(fp_source, fp_target, csv)
# n = [n.outputField.baseName for n in fm.fieldMappings]
fp_out = os.path.join(gdb_pro, 'intermediary/parcels_merged')
arcpy.Merge_management([fp_source, fp_target], fp_out, fm, True)
# dir(fm.fieldMappings[0].outputField)

<Result 'D:\\box_offline\\klamath\\GIS_Request_Tracking\\arcpro_maps\\klamath_d_drive2\\klamath_d_drive2.gdb\\intermediary\\parcels_merged'>

In [197]:
# FIELD MAPPING
# Output field list to csv for formatting
# Created in May 2022; tested in October 2022 with parcels
# output comparison of tables to input into next cell
import copy

fp_in = ['parcels_master_rectified_clipped', 'siskiyou_co_parcels_clipped']
fld, fname = [],[]
df = pd.DataFrame(columns = ['feature_layer','field'])
for idx, f in enumerate(fp_in):
    if idx ==0:
        temp =[fs.name for fs in arcpy.ListFields(f)]
        fld = copy.copy(temp)
        fname = [f]*len(temp)
    else:
        temp = [fs.name for fs in arcpy.ListFields(f)]
        fld += copy.copy(temp)
        fname += [f]*len(temp)                     
    
df['field']=fld
df['feature_layer']=fname
path_csv = os.path.join(inventory_dir, 'parcels_master_siskiyou_join.csv')
df.to_csv(path_csv)


In [214]:
# # Formatting siskiyou for df join then update field

# # Files
# fp_source = r'D:\box_offline\klamath\GIS_Request_Tracking\parcels_easements\parcels_master_rectified.csv'
# fp_target = r'D:\box_offline\klamath\GIS_Request_Tracking\parcels_easements\siskiyou_co_parcels.csv'
# df_source = pd.read_csv(fp_source)
# df_target = pd.read_csv(fp_target)

# Siskiyou apn did NOT have trailing -000 in apn.  Added and joined to master
def format_apn(apn):
    try:
        int(apn[1])
        fmt = '{}-000'.format(apn)
    except ValueError:
        fmt = copy.copy(apn)
    return(fmt)
df_target['apn'] = df_target['apn'].apply(format_apn)
df_target.to_csv(fp_target)

# A) Prepare to subset siskiyou to only matching entries (Note - probably unnecessary)
# Get matching rows between master and siskiyou - 
matching_keys = set(df_source.apn).intersection(set(df_target.apn))
matching_keys = list(matching_keys)

# Use sql string in FC to FC
objid_sql = df_target.set_index('apn').loc[matching_keys,'OID_siskiyou'].to_list()
print(len(objid_sql))
objid_sql = [str(o) for o in objid_sql]
objid_sql = ','.join(objid_sql)
objid_sql = '"FID" IN ({})'.format(objid_sql)

In [193]:
# Subset to matching
wc = objid_sql
fp_out = os.path.join(gdb_pro,'parcels')
arcpy.FeatureClassToFeatureClass_conversion('siskiyou_co_parcels', fp_out, 'siskiyou_co_parcels_matched', where_clause = wc)

<Result 'D:\\box_offline\\klamath\\GIS_Request_Tracking\\arcpro_maps\\klamath_d_drive2\\klamath_d_drive2.gdb\\parcels\\siskiyou_co_parcels_matched'>

In [215]:
# After subsetting siskiyou parcels via matching_keys and formatting (fc_to_fc.keep_fields), 
# fix APN to match parcels master

# # a) Siskiyou apn did NOT have trailing -000 in apn.  Added and joined to master
# fp_target = r'D:\box_offline\klamath\GIS_Request_Tracking\parcels_easements\siskiyou_co_parcels_matched_v2.csv'
# df_target = pd.read_csv(fp_target)

# def format_apn(apn):
#     try:
#         int(apn[1])
#         fmt = '{}-000'.format(apn)
#     except ValueError:
#         fmt = copy.copy(apn)
#     return(fmt)
# df_target['APN'] = df_target['APN'].apply(format_apn)
# df_target.to_csv(fp_target)

# b) update with new apn
utilities.update_field('siskiyou_co_parcels_matched_v2', r'apn_formatted', 1, False, fp_target)

# c) finish work
# remove extra apn field, whatever

In [4]:
# picking this up 2 months later for Local Impact Mitigation Func
# 20230113
import os
import pandas as pd
dir_out = r'C:\Users\UhlmannZachary\Box\GIS\Project_Based\Klamath_River_Renewal_MJA\GIS_Data\data_inventory_and_tracking\working_datasets\parcels'
csv = os.path.join(dir_out, r'parcels_siskiyou_dissolve_sample.csv')
df_diagnose = pd.read_csv(csv)
df_transposed = df_diagnose.T
df_transposed.to_csv(os.path.join(dir_out,r'parcels_sisk_sample_transposed.csv'))

In [12]:
# SQL statements for def queries per layer
import pandas as pd
csv = r'D:\box_offline\klamath\GIS_Request_Tracking\parcels_easements\siskiyou_co_parcels_clipped.csv'
csv = r'D:\box_offline\klamath\GIS_Request_Tracking\parcels_easements\parcels_master_rectified.csv'
csv = r'D:\box_offline\klamath\GIS_Request_Tracking\parcels_easements\siskiyou_co_parcels_matched_v2.csv'
df=pd.read_csv(csv)
# temp = df.groupby('apn').count()
temp = df.groupby('apn').count()
temp = temp[temp['OID_']>1].index
objid_sql = "','".join(temp)
objid_sql = "'{}'".format(objid_sql)
objid_sql

"'003-010-130-000','004-010-010-000','004-020-030-000','004-030-371-000','004-040-060-000','004-040-350-000','004-040-471-000','004-050-010-000','004-050-380-000','004-060-100-000','004-180-280-000','004-190-030-000','004-380-030-000','004-393-291-000','004-393-531-000','004-400-680-000','004-460-261-000','004-460-281-000','004-490-171-000','004-510-331-000','004-530-011-000','004-530-021-000','004-530-031-000','004-540-071-000','004-540-081-000','004-540-091-000','004-540-101-000','004-550-011-000','004-550-031-000','004-550-071-000','004-550-081-000','004-550-091-000','006-020-020-000','006-030-160-000','006-050-060-000','006-060-060-000','006-150-020-000','006-160-030-000','006-160-040-000','006-240-030-000','006-240-310-000','006-250-430-000','006-250-460-000','006-270-010-000','006-280-060-000','006-360-110-000','006-450-040-000','006-450-100-000','006-470-070-000','006-470-150-000','006-480-060-000','006-500-140-000','007-090-090-000','007-140-080-000','007-280-170-000','007-340-

In [8]:
import pandas as pd
csv_limf = r'C:\Users\UhlmannZachary\Box\GIS\Project_Based\Klamath_River_Renewal_MJA\GIS_Request_Tracking\GIS_Requests_from_McMillen\20230112\LIMF_confidential_att_table.csv'
csv_master = r'D:\box_offline\klamath\GIS_Request_Tracking\parcels_easements\parcels_master_rectified.csv'
df_limf = pd.read_csv(csv_limf, index_col='APN')
apns = df_limf.index
# df_master = pd.read_csv(csv_master, index_col = 'apn')

objid_sql = "','".join(apns)
objid_sql = "'{}'".format(objid_sql)
apns

Index(['004-030-421-000', '004-030-481-000', '004-030-661-000',
       '004-040-681-000', '004-393-031-000', '004-393-061-000',
       '004-393-071-000', '004-393-091-000', '004-393-161-000',
       '004-393-171-000', '004-393-231-000', '004-393-241-000',
       '004-393-261-000', '004-393-421-000', '004-393-441-000',
       '004-393-471-000', '004-400-291-000', '004-400-301-000',
       '004-400-371-000', '004-410-251-000', '004-420-021-000',
       '004-420-031-000', '004-420-061-000', '004-420-081-000',
       '004-420-101-000', '004-420-151-000', '004-420-461-000',
       '004-420-471-000', '004-430-041-000', '004-430-121-000',
       '004-430-131-000', '004-430-271-000', '004-430-301-000',
       '004-430-401-000', '004-050-041-000', '004-050-281-000',
       '004-030-460-000'],
      dtype='object', name='APN')

In [36]:
base_dir = r'C:\Users\UhlmannZachary\Box\GIS\Project_Based\Klamath_River_Renewal_MJA\GIS_Request_Tracking\GIS_Requests_from_McMillen\20230112'
csv_source =os.path.join(base_dir, r'LIMF_confidential_att_table.csv')
csv_target = os.path.join(base_dir, 'parcels_limf_v1.csv')
dft = pd.read_csv(csv_target)
dft = dft.set_index('apn')
dfs = pd.read_csv(csv_source)
dfs = dfs.set_index('APN')
df_joined = dft.join(dfs)
df_joined.to_csv(os.path.join(base_dir, 'parcels_limf_v1_joined.csv'))
# df_cols = pd.DataFrame(dft.columns)
# df_cols.to_csv(os.path.join(base_dir, 'limf_join_col_types.csv'))

In [57]:
csv_dict = os.path.join(base_dir, 'limf_join_dict.csv')
df_col_dict = pd.read_csv(csv_dict)
d = dict(zip(df_col_dict.orig, df_col_dict.shp_format))
dt = dict(zip(df_col_dict.shp_format, df_col_dict.typ))

# Renamed columns to shapefile format and saved
# df_joined = df_joined.rename(columns = d)
# df_joined.to_csv(os.path.join(base_dir, 'parcels_limf_v1_joined.csv'))

csv_joined =os.path.join(base_dir, 'parcels_limf_v1_joined.csv')
feat_name = 'parcels_limf_v1'
df_joined = pd.read_csv(csv_joined)
for c in df_joined.columns:
    try:
        if dt[c]=='lng':
            utilities.update_field(feat_name, c, 1, False, csv_joined, data_type = 'Long')
            print(r'LONG ', c)
        if dt[c]=='dbl':
            utilities.update_field(feat_name, c, 1, False, csv_joined, data_type = 'DOUBLE')
            print(r'DOUBLE ',c)
        if dt[c]=='stri':
            utilities.update_field(feat_name, c, 1, False, csv_joined)
            print(r'STRING ', c)
    except KeyError:
        print('!!!!{}'.format(c))


!!!!Unnamed: 0
!!!!OID_
!!!!site_address
!!!!site_csz
!!!!owner_name1
!!!!owner_name2
!!!!owner_address1
!!!!owner_address2
!!!!owner_csz
!!!!care_of
!!!!area_acres
!!!!land_value
!!!!imp_value
!!!!total_value
!!!!use_code
!!!!tax_code
!!!!islistedonexhibite4
!!!!pacificorp_parceltype
!!!!date_lastupdated
!!!!county
!!!!labelclass
!!!!within_floodplain
!!!!khsa_label
!!!!source
!!!!adjacent_property
!!!!adj_reservoir
!!!!OBJECTID
!!!!apn_1
!!!!symbol
!!!!Shape_Length_1
!!!!Shape_Area_1
!!!!OBJECTID_1
!!!!PARCELID
!!!!APN_12
!!!!AssesseeNa
!!!!DBA
!!!!InCareOf
!!!!PSTLADDRES
!!!!PSTLCITY
!!!!PSTLSTATE
!!!!PSTLZIP5
!!!!PSTLZIP4
!!!!SITEADDRES
!!!!CVTTXCD
!!!!USECD
!!!!NGHBRHDCD
!!!!Township
!!!!Range
!!!!Section
!!!!Descr
!!!!STATEDAREA
!!!!LASTUPDATE
!!!!CurrentDoc
!!!!CurrentD_1
!!!!Shape_Leng
!!!!Shape_Length_12
!!!!Shape_Area_12
!!!!label
!!!!Shape_Length
!!!!Shape_Area
ID_limf
['OBJECTID_12', 'Shape', 'apn', 'site_address', 'site_csz', 'owner_name1', 'owner_name2', 'owner_address1',

['OBJECTID_12', 'Shape', 'apn', 'site_address', 'site_csz', 'owner_name1', 'owner_name2', 'owner_address1', 'owner_address2', 'owner_csz', 'care_of', 'area_acres', 'land_value', 'imp_value', 'total_value', 'use_code', 'tax_code', 'islistedonexhibite4', 'pacificorp_parceltype', 'date_lastupdated', 'county', 'labelclass', 'within_floodplain', 'khsa_label', 'source', 'adjacent_property', 'adj_reservoir', 'OBJECTID', 'apn_1', 'symbol', 'Shape_Length_1', 'Shape_Area_1', 'OBJECTID_1', 'PARCELID', 'APN_12', 'AssesseeNa', 'DBA', 'InCareOf', 'PSTLADDRES', 'PSTLCITY', 'PSTLSTATE', 'PSTLZIP5', 'PSTLZIP4', 'SITEADDRES', 'CVTTXCD', 'USECD', 'NGHBRHDCD', 'Township', 'Range', 'Section', 'Descr', 'STATEDAREA', 'LASTUPDATE', 'CurrentDoc', 'CurrentD_1', 'Shape_Leng', 'Shape_Length_12', 'Shape_Area_12', 'label', 'Shape_Length', 'Shape_Area', 'ID_limf', 'risk', 'user_type', 'street', 'city_st']
----Adding field assesss_val ----
----Updating Cursor ---
1. Original Val: None ---- New Val: 90000.0
2. Origina

31. Original Val: None ---- New Val: 21937.5
32. Original Val: None ---- New Val: 27187.5
33. Original Val: None ---- New Val: 20625.0
34. Original Val: None ---- New Val: 55946.4375
35. Original Val: None ---- New Val: 30000.0
36. Original Val: None ---- New Val: 33839.25
37. Original Val: None ---- New Val: 33839.25
38. Original Val: None ---- New Val: 29643.75
39. Original Val: None ---- New Val: 14179.875
40. Original Val: None ---- New Val: 18750.0
41. Original Val: None ---- New Val: 12269.625
DOUBLE  reduce_val1
reduce_val2
['OBJECTID_12', 'Shape', 'apn', 'site_address', 'site_csz', 'owner_name1', 'owner_name2', 'owner_address1', 'owner_address2', 'owner_csz', 'care_of', 'area_acres', 'land_value', 'imp_value', 'total_value', 'use_code', 'tax_code', 'islistedonexhibite4', 'pacificorp_parceltype', 'date_lastupdated', 'county', 'labelclass', 'within_floodplain', 'khsa_label', 'source', 'adjacent_property', 'adj_reservoir', 'OBJECTID', 'apn_1', 'symbol', 'Shape_Length_1', 'Shape_Ar

In [8]:
add_apns = [r'040-250-100', r'041-070-170', r'040-250-100', r'040-230-100', r'103-040-100']
add_apns = [r'{}-000'.format(a) for a in add_apns]
sql_apns = "'"+"', '".join(add_apns) +"'"
sql_apns

"'040-250-100-000', '041-070-170-000', '040-250-100-000', '040-230-100-000', '103-040-100-000'"

In [57]:
import xml.etree.ElementTree as ET
import os
import pandas as pd
#20230124
# Load xml and tree
shp_dir = r'D:\box_offline\klamath\temp\temp_delete25'
xml = os.path.join(shp_dir, r'wetlands_merged_CA_SP1.shp.xml')
shp = os.path.join(shp_dir, r'wetlands_merged_CA_SP1.shp')
tree = ET.parse(xml)
# root is the root ELEMENT of a tree
root = tree.getroot()
dataIdInfo = root.find('dataIdInfo')
eainfo = root.find('eainfo')

In [21]:
# 20230124
# Create xml node dictionary
# 1) save all fields to csv
feat = r'wetlands_merged_CA_SP1'
csv = os.path.join(shp_dir, 'wetlands_merged_CA_SP1_xml_template.csv')
flds = [f.name for f in arcpy.ListFields(shp)]
df_fields = pd.DataFrame(flds)
df_fields.to_csv(csv)

# 2) open csv, add target nodes and vals for key/val and Reload
# dataframe dictionary
df_xml_trans = pd.read_csv(csv, index_col = 'attrlabl')

In [58]:
# 20230124 
# should be valid for fields with FGDC
temp = eainfo.find('detailed')

# If more nodes requiring updates from df add to tgt_nodes list
tgt_nodes = ['attrdef','attrdefs']
attr_child = temp.findall('attr')
for c in attr_child:
    try:
        attrlabl = c.find('attrlabl').text
        for nd in tgt_nodes:
            try:
                v = df_xml_trans.loc[attrlabl, nd]
                # No value from csv/dataframe i.e. no value to replace
                if not pd.isnull(v):
                    c.find(nd).text = df_xml_trans.loc[attrlabl, nd]
                    print('Replaced: {} with: {}'.format(c.find(nd).text, v))
            except KeyError:
                print('Key: {} not present in dataframe'.format(attrlabl))
            except AttributeError:
                # if NoneType --> tgt_node needs to be created and populated in xml
                ET.SubElement(c, nd)
                c.find(nd).text = df_xml_trans.loc[attrlabl, nd]
                print('Created new subelement: {} in {}'.format(nd, attrlabl))
    except KeyError:
        print('The following Key does not have attrlabl: {}'.format(c.tag, c.attr))

tree.write(os.path.join(shp_dir, xml))

Created new subelement: attrdef in WETLAND_ID
Created new subelement: attrdefs in WETLAND_ID
Created new subelement: attrdefs in HYDROLOGY
Created new subelement: attrdef in RESERVOIR
Created new subelement: attrdef in page_num
Created new subelement: attrdef in page_name


In [10]:
import pandas as pd
df = pd.read_csv(r'C:\Users\UhlmannZachary\Box\GIS\Project_Based\Klamath_River_Renewal_MJA\GIS_Request_Tracking\GIS_Requests_from_McMillen\20230112\groundwater\merged_concat.csv')
# idx_dups = df[df.duplicated(subset = ['APN'])].APN
# df['duplicated_apns']=False
# df = df.set_index('APN')
# df.loc[idx_dups, 'duplicated_apns'] = True
# df.to_csv(r'C:\Users\UhlmannZachary\Box\GIS\Project_Based\Klamath_River_Renewal_MJA\GIS_Request_Tracking\GIS_Requests_from_McMillen\20230112\groundwater\merged_concat.csv')

In [13]:
# df_dups = df[df.duplicated_apns]
# df_dups.to_csv(r'C:\Users\UhlmannZachary\Box\GIS\Project_Based\Klamath_River_Renewal_MJA\GIS_Request_Tracking\GIS_Requests_from_McMillen\20230112\groundwater\duplicates_merged_concat.csv')
df['non_pertinent_parcel'] = df.non_pertinent_parcel.replace({np.nan:False})
df.to_csv(r'C:\Users\UhlmannZachary\Box\GIS\Project_Based\Klamath_River_Renewal_MJA\GIS_Request_Tracking\GIS_Requests_from_McMillen\20230112\groundwater\merged_concat.csv')


In [48]:
# Get count of unique owners
import os
# proj_dir = r'C:\Users\UhlmannZachary\Box\GIS\Project_Based\Klamath_River_Renewal_MJA\GIS_Request_Tracking\GIS_Requests_from_McMillen\20230112\groundwater'
# df_pertinent = df[~df['non_pertinent_parcel']]
# df_counts = df_pertinent.groupby(['PSTLADDRES','AssesseeNa']).size()
# df_counts.to_csv(os.path.join(proj_dir, 'unique_owner.csv'))


577

In [37]:
df = pd.read_csv(os.path.join(proj_dir, r'march2020_groundwater_list_zu_20230207.csv'))
apn = [a for a in df.APN.to_list() if not pd.isnull(a)]
apn = ['{}-000'.format(a.strip()) for a in apn]
apn

['004-400-501-000', '004-420-031-000', '041-040-310-000', '102-061-110-000', '004-420-081-000', '004-440-311-000', '004-480-261-000', '102-061-100-000', '041-040-220-000', '102-240-150-000', '004-480-221-000', '041-410-060-000', '102-330-090-000', '041-410-010-000', '102-370-090-000', '041-040-230-000', '102-340-270-000', '004-440-451-000', '004-450-471-000', '004-400-531-000', '004-400-521-000', '102-280-040-000', '102-280-020-000', '004-410-011-000', '004-410-251-000', '102-300-080-000', '102-120-150-000', '102-061-120-000', '041-030-150-000', '102-270-190-000', '103-010-220-000', '102-061-130-000', '102-260-160-000', '004-030-481-000', '041-410-070-000', '041-060-170-000', '041-030-160-000', '041-040-260-000', '004-440-021-000', '004-440-051-000', '102-260-260-000', '102-280-030-000', '102-200-120-000', '102-350-050-000', '102-350-140-000', '004-440-281-000', '041-010-110-000', '103-260-100-000', '004-030-421-000', '004-450-511-000', '104-050-170-000']